# Gene essentiality

In [2]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from bacillusme.util.helper_functions import *

%load_ext autoreload
%autoreload 2
print(cobra.__file__)
print(cobrame.__file__)
print(bacillusme.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/Downloads/cobrapy-0.5.11/cobra/__init__.py
/home/jt/UCSD/cobrame-master/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/bacillusme/__init__.py


In [3]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7f2a27c3e3c8>

In [4]:
with open('../me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)

## Essentiality

M-model

In [5]:
from bacillusme.analysis import essentiality as es

In [6]:
genes = [gene.id for gene in bsub.genes]
essentiality_df_m,flux_responses_m = es.gene_essentiality(bsub,genes, 'm',NP=10,threshold = 0.01,precision=1e-6)

10 threads:   0%|          | 0/844 [00:00<?, ?it/s]

Solving model for base solution


10 threads: 100%|█████████▉| 843/844 [01:00<00:00, 10.29it/s]

Done


In [7]:
flux_responses_m.to_csv('essentiality/essentiality_raw_results.csv')

ME-model

In [ ]:
genes = [gene.id for gene in me.genes]
essentiality_df_me,flux_responses_me = es.gene_essentiality(me,genes,'me',NP=10,threshold = 0.01,precision=1e-6)

Solving model for base solution


10 threads:   6%|▌         | 59/963 [4:13:49<32:04:02, 127.70s/it]

In [ ]:
flux_responses_me.to_csv('essentiality/essentiality_raw_me_results.csv')

### Save combined results

In [ ]:
essentiality_df = essentiality_df_me.join(essentiality_df_m,lsuffix='_me',rsuffix='_m')

In [ ]:
essentiality_df.to_csv('essentiality/essentiality_results.csv')

### Read

In [ ]:
# Read
essentiality_df = pd.read_csv('essentiality_results.csv',index_col=0)
subtiwiki_df = pd.read_csv('subtiwiki_essential.txt',sep='\t')

## M/ME comparison

In [ ]:
essentiality_df[essentiality_df.essentiality_m != essentiality_df.essentiality_me].dropna()

In [ ]:
2